<a href="https://colab.research.google.com/github/sperezconstanzo/DigitalHouse-DataScience-tp/blob/main/desafio_1/modelado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Objetivos
* Correr modelos lineales sobre los set de datos obtenidos en la primera parte de este desafío.
* Elegir distintos hiperparámetros y contrastar cambios.

<a id="section_toc"></a> 
## Tabla de Contenidos

[0 - Imports y declaración de datasets](#section_0)

[1 - Definición de funciones auxiliares](#section_1)

[2 - Definición de hiperparámetros](#section_2)

[3 - Modelado](#section_3)

&nbsp;&nbsp;&nbsp;&nbsp;[3.1 - Capital Federal](#section_3.1)

&nbsp;&nbsp;&nbsp;&nbsp;[3.2 - Argentina](#section_3.2)

<a id="section_0"></a> 
# 0 - Imports y declaración de datasets
[volver a TOC](#section_toc)

Para cargar los datasets de esta práctica necesitamos acceso al directorio de Google Drive donde están los archivos `.csv`. Podemos acceder usando el siguiente link: https://drive.google.com/drive/folders/1Q1iQhLZUujDb0hPUzZ99NNpAygL3uTeB?usp=sharing

Una vez que tenemos acceso, tenemos que agregar un acceso directo de esta carpeta a nuestro Drive.

Finalmente, tenemos que montar el directorio de drive en Google Colab ejecutando el siguiente código y siguiendo las instrucciones.


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
from IPython.display import display
from sklearn import linear_model
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split
import statsmodels.api as sm

pd.options.display.max_columns = None
pd.options.display.max_rows = 40

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
data_path = '/content/drive/MyDrive/Desafio 1/Data/data_clean_final.csv'

data = pd.read_csv(data_path)
data_sin_imputar = data[data.imputados == 0]  # Más adelante vamos a probar cómo cambia el modelo si no usamos datos imputados

print(data.shape)
display(data.head(5))

(82781, 67)


,property_type,place_with_parent_names,region,state_name,place_name_clean,lat,lon,rooms,ammenity_clean,price_aprox_usd_clean,surface_in_m2_clean,price_usd_per_m2_clean,properati_url,description,title,imputados,apartment,house,store,have_amenities,parrilla,cochera,gimnasio,laundry,pileta,quincho,sum,sauna,terraza,rooms_1.0,rooms_2.0,rooms_3.0,rooms_4.0,rooms_5.0,rooms_6.0,rooms_7.0,rooms_8.0,rooms_9.0,rooms_10.0,state_Bs.As. G.B.A. Zona Oeste,state_Bs.As. G.B.A. Zona Sur,state_Buenos Aires Costa Atlántica,state_Buenos Aires Interior,state_Capital Federal,state_Catamarca,state_Chaco,state_Chubut,state_Corrientes,state_Córdoba,state_Entre Ríos,state_Formosa,state_Jujuy,state_La Pampa,state_La Rioja,state_Mendoza,state_Misiones,state_Neuquén,state_Río Negro,state_Salta,state_San Juan,state_San Luis,state_Santa Cruz,state_Santa Fe,state_Santiago Del Estero,state_Tierra Del Fuego,state_Tucumán,barrio_cf_dummy
0,PH,|Argentina|Capital Federal|Mataderos|,BsAs,Capital Federal,Mataderos,-34.661824,-58.508839,2.0,NaN,62000.0,55.0,1127.272727,http://www.properati.com.ar/15bo8_venta_ph_mat...,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Mataderos
1,apartment,|Argentina|Capital Federal|Mataderos|,BsAs,Capital Federal,Mataderos,-34.652262,-58.522982,2.0,NaN,72000.0,55.0,1309.090909,http://www.properati.com.ar/15bod_venta_depart...,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Mataderos
2,apartment,|Argentina|Buenos Aires Costa Atlántica|Mar de...,AMBA,Buenos Aires Costa Atlántica,Mar del Plata,-38.002626,-57.549447,0.0,NaN,64000.0,35.0,1828.571429,http://www.properati.com.ar/15bok_venta_depart...,DEPARTAMENTO CON FANTÁSTICA ILUMINACIÓN NATURA...,DEPTO 2 AMB AL CONTRAFRENTE ZONA CENTRO/PLAZA ...,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN
3,PH,|Argentina|Bs.As. G.B.A. Zona Norte|Vicente Ló...,BsAs,Bs.As. G.B.A. Zona Norte,Vicente López,-34.532957,-58.521782,0.0,NaN,130000.0,106.0,1226.415094,http://www.properati.com.ar/15bor_venta_ph_mun...,MUY BUEN PH AL FRENTE CON ENTRADA INDEPENDIENT...,"MUY BUEN PH AL FRENTE DOS DORMITORIOS , PATIO,...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN
4,apartment,|Argentina|Capital Federal|Belgrano|,BsAs,Capital Federal,Belgrano,-34.559873,-58.443362,1.0,PILETA|LAUNDRY,138000.0,45.0,3066.666667,http://www.properati.com.ar/15bot_venta_depart...,EXCELENTE MONOAMBIENTE A ESTRENAR AMPLIO SUPER...,JOSE HERNANDEZ 1400 MONOAMBIENTE ESTRENAR CAT...,1,1,0,0,1,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Belgrano


In [ ]:
data_capital_path = '/content/drive/MyDrive/Desafio 1/Data/df_capital_federal.csv'

data_capital = pd.read_csv(data_capital_path)
data_capital_sin_imputar = data_capital[data_capital.imputados == 0]  # Más adelante vamos a probar cómo cambia el modelo si no usamos datos imputados

print(data_capital.shape)
display(data_capital.head(5))

(22768, 96)


,property_type,place_with_parent_names,region,state_name,place_name_clean,lat,lon,rooms,ammenity_clean,price_aprox_usd_clean,surface_in_m2_clean,price_usd_per_m2_clean,properati_url,description,title,imputados,apartment,house,store,have_amenities,parrilla,cochera,gimnasio,laundry,pileta,quincho,sum,sauna,terraza,rooms_1.0,rooms_2.0,rooms_3.0,rooms_4.0,rooms_5.0,rooms_6.0,rooms_7.0,rooms_8.0,rooms_9.0,rooms_10.0,barrio_cf_dummy,barrio_Agronomía,barrio_Almagro,barrio_Balvanera,barrio_Barracas,barrio_Barrio Norte,barrio_Belgrano,barrio_Boca,barrio_Boedo,barrio_Caballito,barrio_Catalinas,barrio_Centro / Microcentro,barrio_Chacarita,barrio_Coghlan,barrio_Colegiales,barrio_Congreso,barrio_Constitución,barrio_Flores,barrio_Floresta,barrio_Las Cañitas,barrio_Liniers,barrio_Mataderos,barrio_Monserrat,barrio_Monte Castro,barrio_Nuñez,barrio_Once,barrio_Palermo,barrio_Parque Avellaneda,barrio_Parque Centenario,barrio_Parque Chacabuco,barrio_Parque Chas,barrio_Parque Patricios,barrio_Paternal,barrio_Pompeya,barrio_Puerto Madero,barrio_Recoleta,barrio_Retiro,barrio_Saavedra,barrio_San Cristobal,barrio_San Nicolás,barrio_San Telmo,barrio_Tribunales,barrio_Velez Sarsfield,barrio_Versalles,barrio_Villa Crespo,barrio_Villa Devoto,barrio_Villa General Mitre,barrio_Villa Lugano,barrio_Villa Luro,barrio_Villa Ortuzar,barrio_Villa Pueyrredón,barrio_Villa Real,barrio_Villa Riachuelo,barrio_Villa Santa Rita,barrio_Villa Soldati,barrio_Villa Urquiza,barrio_Villa del Parque
0,PH,|Argentina|Capital Federal|Mataderos|,BsAs,Capital Federal,Mataderos,-34.661824,-58.508839,2.0,NaN,62000.0,55.0,1127.272727,http://www.properati.com.ar/15bo8_venta_ph_mat...,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,Mataderos,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,apartment,|Argentina|Capital Federal|Mataderos|,BsAs,Capital Federal,Mataderos,-34.652262,-58.522982,2.0,NaN,72000.0,55.0,1309.090909,http://www.properati.com.ar/15bod_venta_depart...,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,Mataderos,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,apartment,|Argentina|Capital Federal|Belgrano|,BsAs,Capital Federal,Belgrano,-34.559873,-58.443362,1.0,PILETA|LAUNDRY,138000.0,45.0,3066.666667,http://www.properati.com.ar/15bot_venta_depart...,EXCELENTE MONOAMBIENTE A ESTRENAR AMPLIO SUPER...,JOSE HERNANDEZ 1400 MONOAMBIENTE ESTRENAR CAT...,1,1,0,0,1,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,Belgrano,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,apartment,|Argentina|Capital Federal|Belgrano|,BsAs,Capital Federal,Belgrano,-34.559873,-58.443362,0.0,PILETA|LAUNDRY,195000.0,65.0,3000.000000,http://www.properati.com.ar/15bou_venta_depart...,EXCELENTE DOS AMBIENTES ESTRENAR AMPLIO SUPER...,"JOSE HERNANDEZ 1400 DOS AMBIENTES ESTRENAR ,...",0,1,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Belgrano,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,apartment,|Argentina|Capital Federal|Palermo|Palermo Soho|,BsAs,Capital Federal,Palermo,NaN,NaN,1.0,TERRAZA|PILETA|SAUNA|GIMNASIO,111700.0,50.0,2234.000000,http://www.properati.com.ar/15bp9_venta_depart...,Torre I Mondrian. 3 ambientes con terraza y d...,Vitraux Palermo,0,1,0,0,1,0,0,1,0,1,0,0,1,1,1,0,0,0,0,0,0,0,0,0,Palermo,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


<a id="section_1"></a> 
# 1 - Definición de funciones auxiliares
[volver a TOC](#section_toc)

In [ ]:
def nplog(x, base= np.e):
    """
    Por si queremos calcular logaritmos en alguna base != de 2, e, 10.
    El módulo `math` nativamente tiene `base` como argumento, pero la
    función `log` de `numpy` no, de ahí la necesidad de esta función. Si no se
    especifica base, esta función es equivalente a `np.log(x)`
    """
    return np.log(x) / np.log(base)

In [ ]:
def createModels(x, y, alpha, cv= None, test_size= 0.20, random_state= 10):
    """
    Esta función toma una matriz bidimensional de features (X) y un vector target (y)
    y realiza un split de entrenamiento/testeo sobre los datos. Luego crea los modelos
    Linear, Ridge y Lasso y sobre estos calcula r^2 y mrse.
    Retorna un df con toda la información y los alphas que hayan sido especificados o
    bien el que haya elegido el modelo (si hay más de uno).
    """
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size= test_size, random_state= random_state)
    
    lm = linear_model.LinearRegression(normalize= True)
    model_lr = lm.fit(X_train, y_train)
    pred_lr = model_lr.predict(X_test)
    mse_rl = mean_squared_error(y_test, pred_lr)

    lm_ridge = linear_model.RidgeCV(alphas= alphas, normalize= True, cv= cv) 
    model_ridge = lm_ridge.fit(X_train, y_train)
    pred_ridge = model_ridge.predict(X_test)
    mse_ridge = mean_squared_error(y_test, pred_ridge)
    
    lm_lasso = linear_model.LassoCV(alphas= alphas, normalize= True, cv= cv)
    model_lasso = lm_lasso.fit(X_train, y_train)
    pred_lasso = model_lasso.predict(X_test)
    mse_lasso = mean_squared_error(y_test, pred_lasso)
    
    data = {'Modelo' : ['Linear', 'Ridge', 'Lasso'], 
            'R_square_train' : [model_lr.score(X_train, y_train), model_ridge.score(X_train, y_train), model_lasso.score(X_train, y_train)], 
            'R_square_test' : [model_lr.score(X_test, y_test), model_ridge.score(X_test, y_test), model_lasso.score(X_test, y_test)], 
            'RMSE' : [np.sqrt(mse_rl), np.sqrt(mse_ridge), np.sqrt(mse_lasso)], 
            'Alpha' : ['-', model_ridge.alpha_, model_lasso.alpha_]}
    df = pd.DataFrame(data=data)
    
    return df

In [ ]:
def movecol(df, cols_to_move=[], ref_col='', place='after'):
    """
    Esta función cambia la posición de una o más columnas dentro de un DataFrame;
    más práctico que mover con métodos nativos a `pandas` cuando se tiene un gran
    número de columnas. La operación no se puede realizar `inplace`, sino que
    el df con las columnas relocadas debe guardarse en una variable para uso
    posterior.
    """
    cols = df.columns.tolist()
    
    if place == 'after':
        seg1 = cols[:list(cols).index(ref_col) + 1]
        seg2 = cols_to_move
    if place == 'before':
        seg1 = cols[:list(cols).index(ref_col)]
        seg2 = cols_to_move + [ref_col]
    
    seg1 = [i for i in seg1 if i not in seg2]
    seg3 = [i for i in cols if i not in seg1 + seg2]
    
    return(df[seg1 + seg2 + seg3])

In [ ]:
def stats_function(X, y): 
    """ 
    Esta función utiliza el modulo de statsmodels para realizar la regresión; se le
    debe pasar una matriz bidimensional de features (X) y un vector target (y)
    """
    X = sm.add_constant(X)
    model_stats = sm.OLS(y, X).fit()
    return print(model_stats.summary())

<a id="section_2"></a> 
# 2 - Definición de hiperparámetros
[volver a TOC](#section_toc)

In [ ]:
# Esta lista incluye tres variables cuantitativas pero una es nuestro target (price_usd_per_m2_clean). Otra (price_aprox_usd_clean) deriva de una cuantitativa
# (surface_in_m2_clean) y el target, y la última (rooms) fue convertida en dummy, entonces decidimos dropearlas también.
non_quantitative_features = ["property_type", "place_with_parent_names", "region", "state_name", "place_name_clean", "lat", "lon", "rooms", "ammenity_clean", \
                             "price_aprox_usd_clean", "price_usd_per_m2_clean", "properati_url", "description", "title", "barrio_cf_dummy", "imputados"]

strongest_correlation = ["surface_in_m2_clean", "apartment", "house", "laundry"]  # En la notebook anterior vimos que estas features tienen la correlación más fuerte con el target.

alphas = np.logspace(-8, 1, num= 100)  # Alphas a probar en modelos que soporten este hiperparámetro.

Ajustamos el modelo usando solamente las variables cuantitativas (Para Capital Federal)

<a id="section_3"></a> 
# 3 - Modelado
[volver a TOC](#section_toc)

Tanto para C.A.B.A. como para el país, los modelos corridos fueron idénticos. Probamos con cien alphas distribuídos logaritmicamente entre 1e-8 y 10. Hicimos ocho entrenamientos con LinearRegression, RidgeCV y LassoCV, divididos en dos grupos: para el primer grupo se corrieron los tres modelos sobre estos dataset:
* Dataset completo con variables cuantitativas y dummies y con datos imputados
* Dataset completo con variables cuantitativas y dummies y sin datos imputados
* Dataset parcial de cinco features selectas con datos imputados
* Dataset parcial de cinco features selectas sin datos imputados

Los otros cuatro entrenamientos se corrieron sobre los mismos datasets pero agregando tres columnas:
* ln(superficie)
* log2(superficie)
* log10(superficie)

Al final entrenamos un modelo de `statsmodels` sobre el dataset completo sin imputar.

<a id="section_3.1"></a> 
## 3.1 - Capital Federal
[volver a TOC](#section_toc)

In [ ]:
X = data_capital.drop(columns= non_quantitative_features)
y = data_capital['price_usd_per_m2_clean']

createModels(X, y, alphas, test_size= .20)

,Modelo,R_square_train,R_square_test,RMSE,Alpha
0,Linear,0.395938,0.358149,763.400001,-
1,Ridge,0.395935,0.358074,763.444923,0.000432876
2,Lasso,0.395444,0.357719,763.655740,0.0151991


In [ ]:
X = data_capital_sin_imputar.drop(columns= non_quantitative_features)
y = data_capital_sin_imputar['price_usd_per_m2_clean']

createModels(X, y, alphas)

,Modelo,R_square_train,R_square_test,RMSE,Alpha
0,Linear,0.442752,0.441282,701.259608,-
1,Ridge,0.442750,0.441306,701.244595,0.000231013
2,Lasso,0.442726,0.441361,701.210017,0.00123285


In [ ]:
X = data_capital[strongest_correlation]
y = data_capital['price_usd_per_m2_clean']

createModels(X, y, alphas)

,Modelo,R_square_train,R_square_test,RMSE,Alpha
0,Linear,0.126441,0.113368,897.236438,-
1,Ridge,0.126440,0.113328,897.256484,0.00284804
2,Lasso,0.126441,0.113368,897.236392,1.87382e-05


In [ ]:
X = data_capital_sin_imputar[strongest_correlation]
y = data_capital_sin_imputar['price_usd_per_m2_clean']

createModels(X, y, alphas)

,Modelo,R_square_train,R_square_test,RMSE,Alpha
0,Linear,0.129122,0.153196,863.324403,-
1,Ridge,0.129118,0.153034,863.406557,0.00657933
2,Lasso,0.129122,0.153196,863.324403,1e-08


#### Probando agregar log superficie

In [ ]:
data_capital['surf_m2_log_e'] = np.log(data_capital.loc[ : , "surface_in_m2_clean"])
data_capital['surf_m2_log_2'] = np.log2(data_capital.loc[ : , "surface_in_m2_clean"])
data_capital['surf_m2_log_10'] = np.log10(data_capital.loc[ : , "surface_in_m2_clean"])

data_capital = movecol(data_capital, cols_to_move= ['surf_m2_log_e', 'surf_m2_log_2', 'surf_m2_log_10'], ref_col= 'surface_in_m2_clean')

data_capital_sin_imputar = data_capital[data_capital.imputados == 0]  # Como agregamos columnas al df original, volvemos a generar este df.

In [ ]:
data_capital.columns

Index(['property_type', 'place_with_parent_names', 'region', 'state_name',
       'place_name_clean', 'lat', 'lon', 'rooms', 'ammenity_clean',
       'price_aprox_usd_clean', 'surface_in_m2_clean', 'surf_m2_log_e',
       'surf_m2_log_2', 'surf_m2_log_10', 'price_usd_per_m2_clean',
       'properati_url', 'description', 'title', 'imputados', 'apartment',
       'house', 'store', 'have_amenities', 'parrilla', 'cochera', 'gimnasio',
       'laundry', 'pileta', 'quincho', 'sum', 'sauna', 'terraza', 'rooms_1.0',
       'rooms_2.0', 'rooms_3.0', 'rooms_4.0', 'rooms_5.0', 'rooms_6.0',
       'rooms_7.0', 'rooms_8.0', 'rooms_9.0', 'rooms_10.0', 'barrio_cf_dummy',
       'barrio_Agronomía', 'barrio_Almagro', 'barrio_Balvanera',
       'barrio_Barracas', 'barrio_Barrio Norte', 'barrio_Belgrano',
       'barrio_Boca', 'barrio_Boedo', 'barrio_Caballito', 'barrio_Catalinas',
       'barrio_Centro / Microcentro', 'barrio_Chacarita', 'barrio_Coghlan',
       'barrio_Colegiales', 'barrio_Congreso', '

In [ ]:
X = data_capital.drop(columns= non_quantitative_features)
y = data_capital['price_usd_per_m2_clean']

createModels(X, y, alphas)

,Modelo,R_square_train,R_square_test,RMSE,Alpha
0,Linear,0.426531,0.395487,740.863250,-
1,Ridge,0.427288,0.395202,741.037895,0.000432876
2,Lasso,0.427032,0.394928,741.205577,0.01


In [ ]:
X = data_capital_sin_imputar.drop(columns= non_quantitative_features)
y = data_capital_sin_imputar['price_usd_per_m2_clean']

createModels(X, y, alphas)

,Modelo,R_square_train,R_square_test,RMSE,Alpha
0,Linear,0.447347,0.449917,695.819235,-
1,Ridge,0.447345,0.449847,695.863924,0.000231013
2,Lasso,0.447312,0.449858,695.856705,0.00151991


In [ ]:
X = data_capital[strongest_correlation]
y = data_capital['price_usd_per_m2_clean']

createModels(X, y, alphas)

,Modelo,R_square_train,R_square_test,RMSE,Alpha
0,Linear,0.126441,0.113368,897.236438,-
1,Ridge,0.126440,0.113328,897.256484,0.00284804
2,Lasso,0.126441,0.113368,897.236392,1.87382e-05


In [ ]:
X = data_capital_sin_imputar[strongest_correlation]
y = data_capital_sin_imputar['price_usd_per_m2_clean']

createModels(X, y, alphas)

,Modelo,R_square_train,R_square_test,RMSE,Alpha
0,Linear,0.129122,0.153196,863.324403,-
1,Ridge,0.129118,0.153034,863.406557,0.00657933
2,Lasso,0.129122,0.153196,863.324403,1e-08


<a id="section_3.2"></a> 
## 3.2 - Argentina
[volver a TOC](#section_toc)

In [ ]:
strongest_correlation.append("state_Capital Federal")  # Esta columna está en el df país pero no en el de capital y es una de las que tiene mayor correlación

In [ ]:
X = data.drop(columns= non_quantitative_features)
y = data['price_usd_per_m2_clean']

createModels(X, y, alphas)

,Modelo,R_square_train,R_square_test,RMSE,Alpha
0,Linear,0.387031,0.372382,802.752759,-
1,Ridge,0.387030,0.372372,802.759365,0.000811131
2,Lasso,0.387021,0.372331,802.785566,0.00123285


In [ ]:
X = data_sin_imputar.drop(columns= non_quantitative_features)
y = data_sin_imputar['price_usd_per_m2_clean']

createModels(X, y, alphas)

,Modelo,R_square_train,R_square_test,RMSE,Alpha
0,Linear,0.491876,0.502537,637.481249,-
1,Ridge,0.491875,0.502556,637.468678,0.000811131
2,Lasso,0.491853,0.502589,637.447890,0.00231013


In [ ]:
X = data[strongest_correlation]
y = data['price_usd_per_m2_clean']

createModels(X, y, alphas)

,Modelo,R_square_train,R_square_test,RMSE,Alpha
0,Linear,0.306797,0.296215,850.068822,-
1,Ridge,0.306797,0.296209,850.072535,0.000811131
2,Lasso,0.306797,0.296215,850.068822,4.32876e-08


In [ ]:
X = data_sin_imputar[strongest_correlation]
y = data_sin_imputar['price_usd_per_m2_clean']

createModels(X, y, alphas)

,Modelo,R_square_train,R_square_test,RMSE,Alpha
0,Linear,0.389764,0.399015,700.678245,-
1,Ridge,0.389763,0.399010,700.681271,0.00123285
2,Lasso,0.389764,0.399015,700.678245,1e-08


#### Probando agregar log superficie

In [ ]:
data['surf_m2_log_e'] = np.log(data.loc[ : , "surface_in_m2_clean"])
data['surf_m2_log_2'] = np.log2(data.loc[ : , "surface_in_m2_clean"])
data['surf_m2_log_10'] = np.log10(data.loc[ : , "surface_in_m2_clean"])

data = movecol(data, cols_to_move= ['surf_m2_log_e', 'surf_m2_log_2', 'surf_m2_log_10'], ref_col= 'surface_in_m2_clean')

data_sin_imputar = data[data.imputados == 0]  # Como agregamos columnas al df original, volvemos a generar este df.

In [ ]:
data.columns

Index(['property_type', 'place_with_parent_names', 'region', 'state_name',
       'place_name_clean', 'lat', 'lon', 'rooms', 'ammenity_clean',
       'price_aprox_usd_clean', 'surface_in_m2_clean', 'surf_m2_log_e',
       'surf_m2_log_2', 'surf_m2_log_10', 'price_usd_per_m2_clean',
       'properati_url', 'description', 'title', 'imputados', 'apartment',
       'house', 'store', 'have_amenities', 'parrilla', 'cochera', 'gimnasio',
       'laundry', 'pileta', 'quincho', 'sum', 'sauna', 'terraza', 'rooms_1.0',
       'rooms_2.0', 'rooms_3.0', 'rooms_4.0', 'rooms_5.0', 'rooms_6.0',
       'rooms_7.0', 'rooms_8.0', 'rooms_9.0', 'rooms_10.0',
       'state_Bs.As. G.B.A. Zona Oeste', 'state_Bs.As. G.B.A. Zona Sur',
       'state_Buenos Aires Costa Atlántica', 'state_Buenos Aires Interior',
       'state_Capital Federal', 'state_Catamarca', 'state_Chaco',
       'state_Chubut', 'state_Corrientes', 'state_Córdoba', 'state_Entre Ríos',
       'state_Formosa', 'state_Jujuy', 'state_La Pampa', 's

In [ ]:
X = data.drop(columns= non_quantitative_features)
y = data['price_usd_per_m2_clean']

createModels(X, y, alphas)

,Modelo,R_square_train,R_square_test,RMSE,Alpha
0,Linear,0.449787,0.438501,759.291885,-
1,Ridge,0.449885,0.438699,759.158115,0.000231013
2,Lasso,0.449885,0.438694,759.161337,0.000187382


In [ ]:
X = data_sin_imputar.drop(columns= non_quantitative_features)
y = data_sin_imputar['price_usd_per_m2_clean']

createModels(X, y, alphas)

,Modelo,R_square_train,R_square_test,RMSE,Alpha
0,Linear,0.501129,0.511514,631.703062,-
1,Ridge,0.501273,0.511851,631.484885,0.001
2,Lasso,0.501268,0.511866,631.475151,0.00123285


In [ ]:
X = data[strongest_correlation]
y = data['price_usd_per_m2_clean']

createModels(X, y, alphas)

,Modelo,R_square_train,R_square_test,RMSE,Alpha
0,Linear,0.306797,0.296215,850.068822,-
1,Ridge,0.306797,0.296209,850.072535,0.000811131
2,Lasso,0.306797,0.296215,850.068822,4.32876e-08


In [ ]:
X = data_sin_imputar[strongest_correlation]
y = data_sin_imputar['price_usd_per_m2_clean']

createModels(X, y, alphas)

,Modelo,R_square_train,R_square_test,RMSE,Alpha
0,Linear,0.389764,0.399015,700.678245,-
1,Ridge,0.389763,0.399010,700.681271,0.00123285
2,Lasso,0.389764,0.399015,700.678245,1e-08


In [ ]:
X = data_sin_imputar.drop(columns= non_quantitative_features)
y = data_sin_imputar['price_usd_per_m2_clean']

stats_function(X, y)

                              OLS Regression Results                              
Dep. Variable:     price_usd_per_m2_clean   R-squared:                       0.503
Model:                                OLS   Adj. R-squared:                  0.503
Method:                     Least Squares   F-statistic:                     839.0
Date:                    Mon, 09 Aug 2021   Prob (F-statistic):               0.00
Time:                            23:25:25   Log-Likelihood:            -3.3992e+05
No. Observations:                   43091   AIC:                         6.800e+05
Df Residuals:                       43038   BIC:                         6.804e+05
Df Model:                              52                                         
Covariance Type:                nonrobust                                         
                                         coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------